In [2]:
import pandas as pd

list = []
df = pd.read_csv('FMEL_Dataset.csv')
colums = df.columns
#print(colums)
localGoals = df.localGoals
visitorGoals = df.visitorGoals
concat_goals = pd.concat([localGoals, visitorGoals], axis=1,join_axes=[localGoals.index])
#print(concat_goals)
concat_goals= concat_goals.values
print(len(concat_goals),concat_goals[0][1])

37147 1


In [3]:
concat_goals[0][1]
#mapのイテレータをリストに戻せない
#match_result = []
#match_result = list(map(lambda a: a[0][0]-a[0][1], concat_goals))
#print(match_result)

1

In [58]:
import numpy as np
#左のチームの主観
def distinct(goals):
    match_result = []
    win = 2
    lose = 0
    draw = 1
    #print(goals)
    for a in goals:
        result = a[0] - a[1]
        if result > 0:
            match_result.append(win)
        elif result == 0:
            match_result.append(draw)
        elif result < 0:
            match_result.append(lose)
    return match_result

match_result = distinct(concat_goals)
#リストか配列か
#print(len(distinct(concat_goals)),np.array(match_result))
#

In [59]:
#チーム名のembetting
from sklearn.feature_extraction.text import CountVectorizer
local_team = df.localTeam.values
count = CountVectorizer()
bag = count.fit_transform(local_team)
#print(count.vocabulary_)
embetting_name = bag.toarray()
#embetting_name= embetting_name.tolist()
print(embetting_name.shape)

(37147, 121)


In [60]:
df_round = df['round'].values
print(df_round.shape)
timestamp = df['timestamp'].values
print(timestamp[0])
division = df['division'].values
print(division.shape)

(37147,)
21938400
(37147,)


In [61]:
#df_season = df.season.values.tolist()
#print(df_season)
#df_season = map(lambda a: a.split('-'), df_season)
#print(df_season)

In [62]:
df_season = df.season.values.tolist()
season = [a.split('-') for a in df_season]
for i in range(len(season)):
    season[i] = [int(d) for d in season[i]]
season = np.array(season)
print(season.shape)

(37147, 2)


In [63]:
#date
df_date = df['date'].values

#df_date = list(map(lambda d : d.split('/'),df_date))
#df_date = [d.strip('0') for d in df_date]
df_date = [d.split('/') for d in df_date]

for i in range(len(df_date)):
    df_date[i] = [int(d) for d in df_date[i]]
df_date = np.array(df_date)
print(df_date.shape)
print(len(df_date))

(37147, 3)
37147


In [64]:
#df_new = pd.DataFrame({'season': season, 'division': division, 'round': df_round,'name': embetting_name,'date': df_date, 'timestamp':timestamp})
#df_new[:5]
#df_new_list = df_new.values
#print(df_new_list[:5])
#print(df_new_list[0])

In [65]:
new_df = np.zeros((37147,129),dtype=np.int)

for i in range(len(df_date)):
    new_df[i] =  np.hstack((
        season[i],
        division[i],
        df_round[i],
        embetting_name[i],
        df_date[i],
        timestamp[i],
    ))
    
print(new_df[0])

[    1970       71        1        1        0        0        0        0
        0        0        0        0        0        0        0        0
        0        0        1        0        0        0        0        0
        1        0        0        0        0        0        0        0
        0        0        0        0        0        0        0        1
        0        0        0        0        0        0        0        0
        0        0        0        0        0        0        0        0
        0        0        0        0        0        0        0        0
        0        0        0        0        0        0        0        0
        0        0        0        0        0        0        0        0
        0        0        0        0        0        0        0        0
        0        0        0        0        0        0        0        0
        0        0        0        0        0        0        0        0
        0        0        0        0        0      

In [92]:
import pprint
list = __builtins__.list
def zscore(x):
    xmean = x.mean()
    xstd = np.std(x)
    zscore = (x-xmean)/xstd
    return zscore

def min_max(x):
    min = x.min()
    max = x.max()
    result = (x-min)/(max-min)
    return result
new_df[:,0] = min_max(new_df[:,0])
new_df[:,1] = min_max(new_df[:,1])
new_df[:,125] = min_max(new_df[:,125])
new_df[:,126] = min_max(new_df[:,126])
new_df[:,127] = min_max(new_df[:,127])
new_df[:,128] = min_max(new_df[:,128])
new_df[500]

array([ 0.,  0.,  2., 27.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
      dtype=float32)

In [70]:
#match_result = match_result.astype(np.int32)

In [71]:
new_df = new_df.astype(np.float32)

In [72]:
#試合結果
match_result = np.array(match_result)
print(match_result.shape)

(37147,)


In [73]:
datasets = []
for i,j in zip(new_df,match_result):
    datasets.append((i,j))
print(datasets[0][0].dtype)

float32


In [74]:
#ここからモデル作り
from chainer.datasets import split_dataset_random
train_number = int(len(datasets) * 0.6)
train_and_valid = int(len(datasets) * 0.8)
test = int(len(datasets) * 0.2)
train_valid_datasets = datasets[0:train_and_valid]
train = train_valid_datasets[0:train_number]
validation = train_valid_datasets[train_number+1:len(train_valid_datasets)]
test = datasets[train_and_valid+1:len(datasets)]

Iteratorの作成

Iterator:データセットから決まった数のデータとラベルを取得し、それらを束ねてミニバッチを作成する機能を持ったもの。

SerialIterator:データセットの中のデータを順番に取り出してくるもっともシンプルなIterator

chainはパラメータを持つ層(Link)をまとめておくためのクラス。
モデルのパラメータの更新はOptimizerという機能が担う。
chain.params()で更新されるパラメータ一覧が取得できる。

In [76]:
from chainer import iterators

batchsize = 128

train_iter = iterators.SerialIterator(train, batchsize)
valid_iter = iterators.SerialIterator(
    validation, batchsize, repeat=False, shuffle=False)
test_iter = iterators.SerialIterator(
    test, batchsize, repeat=False, shuffle=False)

In [78]:
import chainer
import chainer.links as L
import chainer.functions as F

class MLP(chainer.Chain):
    def __init__(self, n_mid_units=100, n_out=3):
        super(MLP, self).__init__()
        
        with self.init_scope():
            self.l1 = L.Linear(129, n_mid_units)
            self.l2 = L.Linear(n_mid_units, n_mid_units)
            self.l3 = L.Linear(n_mid_units, n_out)
    
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)
    
gpu_id = -1  # CPUを用いる場合は、この値を-1にしてください

net = MLP()

if gpu_id >= 0:
    net.to_gpu(gpu_id)

In [81]:
from chainer import optimizers

optimizer = optimizers.SGD(lr=0.01).setup(net)

In [84]:
import numpy as np
from chainer.dataset import concat_examples
from chainer.cuda import to_cpu

max_epoch = 10

while train_iter.epoch < max_epoch:

    # ---------- 学習の1イテレーション ----------
    train_batch = train_iter.next()
    x, t = concat_examples(train_batch, gpu_id)
    
    #print('x:{}'.format(x.shape))
    
    # 予測値の計算
    y = net(x)
    #print("t{}".format(t.shape))

    # ロスの計算
    loss = F.softmax_cross_entropy(y, t)

    # 勾配の計算
    net.cleargrads()
    loss.backward()

    # パラメータの更新
    optimizer.update()
    # --------------- ここまで ----------------

    # 1エポック終了ごとにValidationデータに対する予測精度を計測
    if train_iter.is_new_epoch:  # 1 epochが終わったら

        # ロスの表示
        print('epoch:{:02d} train_loss:{:.04f} '.format(
            train_iter.epoch, float(to_cpu(loss.data))), end='')

        valid_losses = []
        valid_accuracies = []
        while True:
            valid_batch = valid_iter.next()
            x_valid, t_valid = concat_examples(valid_batch, gpu_id)

            # Validationデータをforward
            with chainer.using_config('train', False), \
                    chainer.using_config('enable_backprop', False):
                y_valid = net(x_valid)

            # ロスを計算
            loss_valid = F.softmax_cross_entropy(y_valid, t_valid)
            valid_losses.append(to_cpu(loss_valid.array))

            # 精度を計算
            accuracy = F.accuracy(y_valid, t_valid)
            accuracy.to_cpu()
            valid_accuracies.append(accuracy.array)

            if valid_iter.is_new_epoch:
                valid_iter.reset()
                break

        print('val_loss:{:.04f} val_accuracy:{:.04f}'.format(
            np.mean(valid_losses), np.mean(valid_accuracies)))

# テストデータでの評価
test_accuracies = []
while True:
    test_batch = test_iter.next()
    x_test, t_test = concat_examples(test_batch, gpu_id)

    # テストデータをforward
    with chainer.using_config('train', False), \
            chainer.using_config('enable_backprop', False):
        y_test = net(x_test)

    # 精度を計算
    accuracy = F.accuracy(y_test, t_test)
    accuracy.to_cpu()
    test_accuracies.append(accuracy.array)

    if test_iter.is_new_epoch:
        test_iter.reset()
        break

print('test_accuracy:{:.04f}'.format(np.mean(test_accuracies)))

epoch:06 train_loss:1.0312 val_loss:1.0797 val_accuracy:0.4525
epoch:07 train_loss:1.0052 val_loss:1.1031 val_accuracy:0.4525
epoch:08 train_loss:0.9869 val_loss:1.0966 val_accuracy:0.4525
epoch:09 train_loss:1.0025 val_loss:1.0924 val_accuracy:0.4525
epoch:10 train_loss:0.9947 val_loss:1.0815 val_accuracy:0.4525
test_accuracy:0.4783
